In [ ]:
#conda list
#!pip install pmdarima --user

In [ ]:
import pandas as pd
import numpy as np
from pmdarima.arima import auto_arima
from datetime import datetime

# Q1

In [ ]:
df1 = pd.read_csv("res/Historical_Data1.csv")
df1.head()

In [ ]:
df1.dtypes

In [ ]:
df1['Date'] = df1['Date'].apply(lambda x:pd.to_datetime(str(x), format='%Y%m%d'))
df1.head()

In [ ]:
df1.dtypes

In [ ]:
total_sales_day = df1.groupby('Date')['Sold_Units'].sum().reset_index()
total_sales_day.head()

In [ ]:
#res1_num_days_sold3 = np.where(total_sales_day['Sold_Units'] > 3)
res1_num_days_sold3 = pd.Series(total_sales_day['Sold_Units'] > 3).sum()
print(res1_num_days_sold3)

In [ ]:
#country_sales = pd.DataFrame(df1.groupby(['Country_Code', df1['Date'].dt.strftime('%B')])['Sold_Units'].sum().reset_index())
country_sales = df1.groupby(['Country_Code', df1['Date'].dt.strftime('%B')])['Sold_Units'].sum().reset_index()

In [ ]:
#fr_sales = country_sales[country_sales['Country_Code']=='FR']
#fr_sales_aug = fr_sales[fr_sales['Date']=='August']
res2_fr_sales_aug = country_sales[country_sales['Country_Code']=='FR'][country_sales['Date']=='August'].iloc[0,2]
print(res2_fr_sales_aug)
#res2_fr_sales_aug = fr_sales_aug.iloc[0,2]
#res2_fr_sales_aug

In [ ]:
res3_AT_sales = country_sales[country_sales['Country_Code']=='AT']['Sold_Units'].sum()
#res3_AT_sales = AT_sales['Sold_Units'].sum()
print(res3_AT_sales)

In [ ]:
result = [res1_num_days_sold3, res2_fr_sales_aug, res3_AT_sales]
result = pd.DataFrame(result)
result.to_csv('output/output.csv', header=False, index=False)

# Q2

In [ ]:
df2=pd.read_csv("res/Historical_Data1.csv")
df2['Date'] = df2['Date'].apply(lambda x:pd.to_datetime(str(x), format='%Y%m%d').date())
df2.head()

In [ ]:
country_group = pd.DataFrame(df2.groupby(['Country_Code','Date'])['Sold_Units'].sum().reset_index())
country_group.head()

In [ ]:
#countries = country_group['Country_Code'].unique()
#countries

In [ ]:
#country_group.set_index(country_group['Date'], freq)

In [ ]:
frames=[]
for loc in country_group['Country_Code'].unique():
    frames.append(pd.DataFrame(data={'Country_Code':loc, 'Date':pd.date_range(start=min(country_group.Date), end=max(country_group.Date)), 'Sold_Units':0}))
df_combined = pd.concat(frames)
df_combined.set_index(['Country_Code', 'Date'], inplace=True)
df_combined.update(country_group.set_index(['Country_Code', 'Date']))
df_combined.reset_index(inplace=True)
df_combined.head()
    

In [ ]:
FR_sales = df_combined[df_combined['Country_Code']=='FR'][df_combined['Sold_Units']>0].sort_values(by='Date').iloc[0,1]
res1_FR_sales = FR_sales.date()
print(res1_FR_sales)

In [ ]:
AT_no_sales = df_combined[df_combined['Country_Code']=='AT'][df_combined['Sold_Units']==0].count()
res2_AT_no_sales = AT_no_sales[0]
print(res2_AT_no_sales)

# Q3

In [ ]:
df3 = pd.read_csv("res/Historical_Data1.csv")
df3.head()

In [ ]:
df3['Date'] = df3['Date'].apply(lambda x:pd.to_datetime(str(x), format='%Y%m%d').date())
country_group = pd.DataFrame(df3.groupby(['Country_Code','Date'])['Sold_Units'].sum().reset_index())

In [ ]:
#df3.set_index('Date', inplace=True)

In [ ]:
#df3.sort_index(inplace=True)
#df3.head()


In [ ]:
frames=[]
for loc in country_group['Country_Code'].unique():
    frames.append(pd.DataFrame(data={'Country_Code':loc, 'Date':pd.date_range(start=min(country_group.Date), end=max(country_group.Date)), 'Sold_Units':0}))
df_combined = pd.concat(frames)
df_combined.set_index(['Country_Code', 'Date'], inplace=True)
df_combined.update(country_group.set_index(['Country_Code', 'Date']))
df_combined.reset_index(inplace=True)
df_combined.set_index('Date', inplace=True)
df_combined.sort_index(inplace=True)
df_combined.head()

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
result = []
for loc in df_combined['Country_Code'].unique():
    df4 = df_combined[df_combined['Country_Code']==loc]
    X = df4['Country_Code'].values
    train_size=int(len(X)-10)
    train, test = df4.iloc[0:train_size,1], df4.iloc[train_size:len(X)]
    
    model = auto_arima(train, trace=True, suppress_warnings=True, error_action='ignore')
    model.fit(train)
    
    prediction = pd.DataFrame(model.predict(),index=test.index)
    prediction.columns=['predicted_units']
    test['predicted_units'] = prediction['predicted_units']
    mae = round(mean_absolute_error(test['Sold_Units'], test['predicted_units']),3)
    result.append(mae)
    print(mae)
    
    

In [ ]:
X = df4['Country_Code'].values
train_size=int(len(X)-10)
train, test = df4.iloc[0:train_size,1], df4.iloc[train_size:len(X)]
model = auto_arima(train, trace=True, suppress_warnings=True, error_action='ignore')
model.fit(train)

prediction = pd.DataFrame(model.predict(),index=test.index)
prediction.columns=['predicted_units']
test['predicted_units'] = prediction['predicted_units']
test

In [ ]:
mae = round(mean_absolute_error(test['Sold_Units'], test['predicted_units']),3)
mae

In [98]:
result = [round(result[0], 2), round(result[1], 3), round(result[2], 2), round(result[3], 2)]
result

[2.09, 1.759, 2.71, 4.15]